# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

from os import listdir, walk
from os.path import isfile, join
from sklearn.preprocessing import MinMaxScaler

# Load Raw Data

In [2]:
%run ../ddc/load_dataset.ipynb

In [3]:
subj_range = np.hstack((np.arange(2001,2002),np.arange(3001,3006)))

all_patients = [str(i) for i in subj_range]

In [4]:
datapath = '../../DDC_Data/'
basepath = '../../'

df_all_p = pd.DataFrame()

for subject_id in all_patients:
    print("Loading {0}'s data".format(subject_id))
    
    acc_filepath = mypath + subject_id + '/' + subject_id + '-log_acc.csv'
    df_raw = pd.read_csv(acc_filepath, header=None, names=['x','y','z','timestamp'])

    df_timer, rec_date, start_time, end_time = load_timer(subject_id)
    df_filt = load_acc(subject_id, rec_date, start_time, end_time)
    df_hr = load_hr(subject_id, rec_date, start_time, end_time)

    df1 = merge_acc_and_hr(df_filt, df_hr)
    
    cols = ['x','y','z']
    xyz_ = df1[cols].to_dict('split')['data']
    xyz_new = MinMaxScaler().fit_transform(xyz_)
    
    for i in range(len(cols)):
        df1[cols[i]] = pd.Series(xyz_new.transpose()[i])

    X_i_p = np.array(df1[cols].to_dict(orient='split')['data'])
    subj_i_p = np.array([subject_id for i in range(X_i_p.shape[0])])
    
    df_all_p = df_all_p.append(df1, sort=False)
    
print('Finished Loading')

Loading 2001's data
Loading 3001's data
Loading 3002's data
Loading 3003's data
Loading 3004's data
Loading 3005's data
Finished Loading


In [5]:
df_all_p_sorted = df_all_p.copy()
df_all_p_sorted = df_all_p_sorted.reset_index(drop=True)

# Label List

In [8]:
%run ../ddc/preprocessing.ipynb

In [18]:
df_label = pd.DataFrame({
    'label': [i for i in range(len(label_list))],
    'activity name': label_list
})

# Store Cleaned Data in CSV

In [16]:
csv_path = datapath + 'cleaned/cleaned_data_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
df_all_p_sorted.to_csv(csv_path)

In [19]:
csv_path = datapath + 'cleaned/label_names.csv'
df_label.to_csv(csv_path)